In [290]:
%load_ext autoreload
%autoreload 2

from dataloader import FakenNewsNet, WICO
import tensorflow as tf
from model import GNNCL
from tqdm import tqdm
import numpy as np
from utils import *

from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import binary_accuracy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [306]:
crossval_results = []
n_splits = 1
batch_size = 16

for i in range(n_splits):
    print(f"Experiment {i+1} of {n_splits}")
    train_loader, test_loader = random_split(data, train_epochs=10, batch_size=batch_size)

    train_loss, train_acc, test_loss, test_acc = evaluate(train_loader, test_loader,
                                                 channels=64, batch_size=batch_size,
                                                 dataset="FakeNewsNet")

    crossval_results.append((train_loss, train_acc, test_loss, test_acc))

Experiment 1 of 1
Ep. 1 - Loss: 7.307. Acc: 0.523
Ep. 2 - Loss: 7.247. Acc: 0.527
Ep. 3 - Loss: 7.367. Acc: 0.520
Ep. 4 - Loss: 7.247. Acc: 0.527
Ep. 5 - Loss: 7.397. Acc: 0.518
Ep. 6 - Loss: 7.427. Acc: 0.516
Ep. 7 - Loss: 7.247. Acc: 0.527
Ep. 8 - Loss: 7.427. Acc: 0.516
Ep. 9 - Loss: 7.187. Acc: 0.531
Done. Test loss: 7.858. Test acc: 0.487


In [301]:
def train_step(model, loss_fn, optimizer, inputs, target):
    with tf.GradientTape() as tape:
        pred = tf.reshape(model(inputs[:2]), (-1,1))
        loss = loss_fn(target, pred)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    acc = tf.reduce_mean(binary_accuracy(target, pred))
    return loss, acc

In [305]:
def evaluate(train_loader, test_loader, num_classes=1, lr=0.01, channels=64,
             batch_size=32, dataset="FakeNewsNet"):
    if dataset == "WICO":
        num_classes = 3
        
    model = GNNCL(num_classes=num_classes, channels=channels, batch_size=batch_size)
    loss_fn = BinaryCrossentropy()
    optimizer = Adam(learning_rate=lr)

    epoch = step = 0
    train_losses = []
    train_accuracies = []

    for inputs, target in train_loader:
        if target.shape[0] == batch_size:
            step += 1
            loss, acc = train_step(model, loss_fn, optimizer, inputs, target)
            train_losses.append(loss)
            train_accuracies.append(acc)

        if step == train_loader.steps_per_epoch:
            step = 0
            epoch += 1
            print("Ep. {} - Loss: {:.3f}. Acc: {:.3f}".format(epoch, np.mean(train_losses), np.mean(train_accuracies)))
            train_losses = []
            train_accuracies = []
    
    test_losses = []
    test_accuracies = []

    for inputs, target in test_loader:
        if target.shape[0] == batch_size:
            pred = tf.reshape(model(inputs[:2], training=False), (-1,1))
            test_losses.append(loss_fn(target, pred))
            acc = (pred == target)
            test_accuracies.append(acc.numpy().mean())
        
    print("Done. Test loss: {:.3f}. Test acc: {:.3f}".format(np.mean(test_losses), np.mean(test_accuracies)))

    return train_losses, train_accuracies, test_losses, test_accuracies

In [297]:
data = FakenNewsNet("./dataset/FakeNewsNet/politifact/")
#data = WICO()